In [1]:
from qiskit_ibm_provider import IBMProvider
from qiskit import Aer
from qiskit import QuantumCircuit, transpile, ClassicalRegister
from qiskit.circuit import Operation
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit.providers.models import BackendProperties
from qiskit.tools.visualization import plot_histogram
from qiskit.visualization import dag_drawer
from qiskit.converters import *
from qiskit.dagcircuit import dagnode
import os
import pickle
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm
from scipy import stats
from collections import defaultdict
with open("API_KEY.txt","r") as file:
    key = file.read()
provider = IBMProvider(token=key)
backends = provider.backends(filters=lambda x: x if "simulator" not in x.name else None)
backends = sorted(backends,key=lambda x: x.name)
sim_ideal = AerSimulator(seed_simulator=42)
backends

[<IBMBackend('ibm_lagos')>,
 <IBMBackend('ibm_nairobi')>,
 <IBMBackend('ibm_perth')>,
 <IBMBackend('ibmq_belem')>,
 <IBMBackend('ibmq_jakarta')>,
 <IBMBackend('ibmq_lima')>,
 <IBMBackend('ibmq_manila')>,
 <IBMBackend('ibmq_quito')>]

In [2]:
def read_backend_data(filename):
    file = open(filename,"rb")
    data = pickle.load(file)
    file.close()
    return data

def read_and_transpile_circuit(filename,hardware,inputvalue=None):
    if inputvalue!=None:
        if inputvalue=="-1":
            circ = QuantumCircuit.from_qasm_file(filename)
            inputvalue = "1"*len(circ.qubits)
            
        circ = QuantumCircuit.from_qasm_file(filename)

        if len(inputvalue)<len(circ.qubits):
            padding = "0"*(len(circ.qubits)-len(inputvalue))
            inputvalue = padding+inputvalue
            
        full_circ = QuantumCircuit()
        for reg in circ.qregs:
            full_circ.add_register(reg)
        for reg in circ.cregs:
            full_circ.add_register(reg)
        for i,inp in enumerate(inputvalue):
            if inp=="1":
                full_circ.x(i)

        full_circ = full_circ.compose(circ)
        circ_trans = transpile(full_circ, hardware,optimization_level=0)
    else:
        circ = QuantumCircuit.from_qasm_file(filename)
        circ_trans = transpile(circ, hardware,optimization_level=0)
    return circ, circ_trans

def convert_circuit_to_graph(circ_tanspiled):
    graph = circuit_to_dag(circ_tanspiled)
    return graph


def get_inverse_circuit(circ_trans,graph):
    measurement_qubits = [(circ_trans.find_bit(node.qargs[0]).index,circ_trans.find_bit(node.cargs[0]).index) for node in graph.op_nodes() if node.op.name=="measure"]
    circ_trans_reverse = circ_trans.remove_final_measurements(inplace=False)
    inv = circ_trans_reverse.inverse()
    circ_trans_reverse = circ_trans_reverse.compose(inv)
    circ_trans_reverse.add_register(ClassicalRegister(circ_trans.num_clbits))
    circ_trans_reverse.barrier(circ_trans_reverse.qubits)
    for mindex in measurement_qubits:
        circ_trans_reverse.measure(mindex[0],mindex[1])
    
    return circ_trans_reverse

def get_circuit_measurements(circ_trans,graph):
    measurement_qubits = [(circ_trans.find_bit(node.qargs[0]).index,circ_trans.find_bit(node.cargs[0]).index) for node in graph.op_nodes() if node.op.name=="measure"]
    return measurement_qubits
    
def execute_with_repitions(circuit, simulator ,repitions=50):
    circ_list = [circuit for x in range(repitions)]
    states = simulator.run(circ_list,shots=1024).result().get_counts()
    return [dict([(k,v/1024) for k,v in x.items()]) for x in states]


def split_circuit(circuit, start, end, graph):
    nq = len(circuit.qubits)
    qc2 = QuantumCircuit(nq)
    for x in circuit[start:end]:
        qc2.append(x[0], x[1])
    
    inv = qc2.inverse()
    qc2 = qc2.compose(inv)
    measurement_qubits = get_circuit_measurements(circuit,graph)
    qc2.add_register(ClassicalRegister(circuit.num_clbits))
    for mindex in measurement_qubits:
        qc2.measure(mindex[0],mindex[1])
    return qc2

def split_circuit_to_percentile(circ_trans,graph):
    p1 = int(np.percentile([x for x in range(len(circ_trans))],25))
    p2 = int(np.percentile([x for x in range(len(circ_trans))],50))
    p3 = int(np.percentile([x for x in range(len(circ_trans))],75))
    qc1 = split_circuit(circ_trans,0,p1,graph)
    qc2 = split_circuit(circ_trans,p1,p2,graph)
    qc3 = split_circuit(circ_trans,p2,p3-1,graph) 
    return qc1,qc2,qc3

def get_gate_info(circ, dag):
    
    features = {"Num_1Q_Gates":0,
                "Num_2Q_Gates":0}
    
    for node in dag.nodes():
            try:
                if node.qargs:
                    if node.name=="barrier" or node.name=="measure":
                        continue
                    if len(node.qargs)==1:
                        features["Num_1Q_Gates"]+=1
                    else:
                        features["Num_2Q_Gates"]+=1
            except:
                pass
    return features
    
def HellingerDistance(p, q):
    n = len(p)
    sum_ = 0.0
    for i in range(n):
        sum_ += (np.sqrt(p[i]) - np.sqrt(q[i]))**2
    result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum_)
    return result
    
def generate_state_features(state, circ, circ_trans, sim_noise, selected_backend):
    # conver transpiled circuit to DAG
    graph = convert_circuit_to_graph(circ_tanspiled=circ_trans)
    graph2 = convert_circuit_to_graph(circ)
    # divide transpiled circuit to 3 percentiles inverted circuit
    qc1,qc2,qc3 = split_circuit_to_percentile(circ,graph2)
    # get inverse transpiled circuit
    circ_trans_inverted = get_inverse_circuit(circ_trans=circ,graph=graph2)
    # execute all circuits on noise backend
    noise_state_runs = execute_with_repitions(circ_trans,simulator=sim_noise,repitions=2)
    
    circ_trans_inverted = transpile(circ_trans_inverted,selected_backend)
    qc1 = transpile(qc1,selected_backend)
    qc2 = transpile(qc2,selected_backend)
    qc3 = transpile(qc3,selected_backend)
    
    inverted_state_runs = execute_with_repitions(circ_trans_inverted,simulator=sim_noise,repitions=2)[0]
    qc1_state_runs = execute_with_repitions(qc1,simulator=sim_noise,repitions=2)[0]
    qc2_state_runs = execute_with_repitions(qc2,simulator=sim_noise,repitions=2)[0]
    qc3_state_runs = execute_with_repitions(qc3,simulator=sim_noise,repitions=2)[0]
    # generate all features
    
    state_features = get_gate_info(circ,graph)
    try:
        observed_prob_25 = np.round(np.percentile([x[state] for x in noise_state_runs if state in list(x.keys())],25),5)
        observed_prob_50 = np.round(np.percentile([x[state] for x in noise_state_runs if state in list(x.keys())],50),5)
        observed_prob_75 = np.round(np.percentile([x[state] for x in noise_state_runs if state in list(x.keys())],75),5)
        odds_ratio = [x[state]/(1-x[state]) for x in noise_state_runs if state in list(x.keys())]
        Avg_odds_ratio = np.round(max(odds_ratio)/min(odds_ratio),5)
        #Avg_odds_ratio = np.round(np.mean([x[state]/(1-x[state]) for x in noise_state_runs if state in list(x.keys())]),5)
        Avg_observed_prob = np.round(np.max([x[state] for x in noise_state_runs if state in list(x.keys())]),5)
    except:
        observed_prob_25 = 0.0
        observed_prob_50 = 0.0
        observed_prob_75 = 0.0
        Avg_odds_ratio =  0.0
        Avg_observed_prob = 0.0
    
    try:
        P = []
        Q = []
        for inv_keys in inverted_state_runs.keys():
            if sum([int(x) for x in inv_keys if x!=' '])==0:
                P.append(1)
            else:
                P.append(0)
            Q.append(inverted_state_runs[inv_keys])
        
        Avg_inverted_error = HellingerDistance(P,Q)
        
        P = []
        Q = []
        for inv_keys in qc1_state_runs.keys():
            if sum([int(x) for x in inv_keys if x!=' '])==0:
                P.append(1)
            else:
                P.append(0)
            Q.append(qc1_state_runs[inv_keys])
        
        Avg_inverted_error_25 = HellingerDistance(P,Q)
        
        P = []
        Q = []
        for inv_keys in qc2_state_runs.keys():
            if sum([int(x) for x in inv_keys if x!=' '])==0:
                P.append(1)
            else:
                P.append(0)
            Q.append(qc2_state_runs[inv_keys])
        
        Avg_inverted_error_50 = HellingerDistance(P,Q)
        
        P = []
        Q = []
        for inv_keys in qc3_state_runs.keys():
            if sum([int(x) for x in inv_keys if x!=' '])==0:
                P.append(1)
            else:
                P.append(0)
            Q.append(qc3_state_runs[inv_keys])
        
        Avg_inverted_error_75 = HellingerDistance(P,Q)
        
    except Exception as e:
        print(inverted_state_runs)
        raise(e)
    state_weight = len([x for x in state if x=="1"])
    circuit_depth = circ_trans.depth()
    circuit_width = circ_trans.width()
    
    state_features["circuit_depth"] = circuit_depth
    state_features["circuit_width"] = circuit_width
    state_features["observed_prob_25"] = np.round(observed_prob_25*100,2)
    state_features["observed_prob_50"] = np.round(observed_prob_50*100,2)
    state_features["observed_prob_75"] = np.round(observed_prob_75*100,2)
    state_features["Avg_odds_ratio"] = np.round(Avg_odds_ratio,2)
    state_features["Avg_inverted_error"] = np.round(Avg_inverted_error*100,2)
    state_features["state_weight"] = state_weight
    #state_features["Avg_observed_prob"] = Avg_observed_prob
    state_features["Avg_inverted_error_25"] = np.round(Avg_inverted_error_25*100,2)
    state_features["Avg_inverted_error_50"] = np.round(Avg_inverted_error_50*100,2)
    state_features["Avg_inverted_error_75"] = np.round(Avg_inverted_error_75*100,2)
    
    return state_features

        
def generate_test_data_real():
    for backend in tqdm(backends,position=0):
        noise_model = NoiseModel.from_backend(backend)
        sim_noise = AerSimulator(noise_model=noise_model)

        circuitfiles = sorted(os.listdir("./testing_circuits/"))
        np.random.shuffle(circuitfiles)
        for circuitfile in circuitfiles:
            circ,circ_trans = read_and_transpile_circuit(os.path.join("./testing_circuits/",circuitfile),backend)
            graph = convert_circuit_to_graph(circ_trans)
            ideal = sim_ideal.run(circ_trans,shots=1024).result().get_counts()
            noisy = sim_noise.run(circ_trans,shots=1024).result().get_counts()
            dataframe = pd.DataFrame()
            for state in noisy.keys():
                try:
                    state_features = generate_state_features(state, circ, circ_trans, sim_noise,backend)
                    if state in list(ideal.keys()):
                        state_features["target"] = np.round((ideal[state]/1024)*100,2)
                    else:
                        state_features["target"] = 0.0
                    dataframe = pd.concat([dataframe,pd.DataFrame.from_records(state_features,index=[0])],ignore_index=0)
                    filename = f"/{circuitfile.split('_')[0]}_{backend.name}"
                    dataframe.to_csv(f"./real_circuits{filename}.csv",index=False)
                except Exception as e:
                    print("exception")
                    print(e)
                    continue
        del sim_noise

# Generating data

In [3]:
if __name__ == "__main__":
    generate_test_data_real()

  0%|          | 0/8 [00:00<?, ?it/s]